# Preprocess POWGEN files

Loads test data files with Mantid and stores them as Scipp HDF5 files to be used in the example workflow.

In [ ]:
import scipp as sc
import scippneutron as scn
import plopp as pp

import ess
from ess.diffraction.external import load_calibration
from ess.diffraction import powder
from ess import diffraction
from ess.external import powgen

In [ ]:
ess.logging.configure_workflow('powgen_preprocess', filename=None)

## Sample

In [ ]:
sample = scn.load(powgen.data.mantid_sample_file(),
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={"LoadMonitors": False})

In [ ]:
sample

In [ ]:
sample.coords['gd_prtn_chrg'] = sample.attrs.pop('gd_prtn_chrg')
sample.coords.set_aligned('gd_prtn_chrg', False)
sample.attrs.clear()
sample_dg = sc.DataGroup({
    'data': sample,
    'detector_info': sample.coords.pop('detector_info').value,
})

In [ ]:
sample_dg

In [ ]:
sample_dg.save_hdf5('PG3_4844_event.h5')

In [ ]:
sc.io.load_hdf5('PG3_4844_event.h5')

## Vana

In [ ]:
vana = scn.load(powgen.data.mantid_vanadium_file(),
                advanced_geometry=False,
                load_pulse_times=True,
                mantid_args={"LoadMonitors": False})

In [ ]:
vana.coords['gd_prtn_chrg'] = vana.attrs.pop('gd_prtn_chrg')
vana.coords.set_aligned('gd_prtn_chrg', False)
vana_dg = sc.DataGroup({
    'data': vana,
    'proton_charge': vana.attrs.pop('proton_charge').value.rename(time='pulse_time')
})
vana.attrs.clear()

In [ ]:
vana_dg

In [ ]:
vana_dg['data'].bins.constituents['data']

In [ ]:
vana_dg['data'].bins.constituents['data'].coords['tof']

In [ ]:
vana_dg.save_hdf5('PG3_4866_event.h5')

In [ ]:
sc.io.load_hdf5('PG3_4866_event.h5')

## Calibration

In [ ]:
cal = load_calibration(
    powgen.data.mantid_calibration_file(),
    instrument_filename='POWGEN_Definition_2011-02-25.xml',
)

In [ ]:
cal

In [ ]:
cal.save_hdf5('PG3_FERNS_d4832_2011_08_24.h5')

In [ ]:
sc.io.load_hdf5('PG3_FERNS_d4832_2011_08_24.h5')